<a href="https://colab.research.google.com/github/Korolev-Sergey/Best_Hack_2021_final/blob/main/BunchofGeeks_pulse.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2     # for capturing videos
import math   # for mathematical operations
import matplotlib.pyplot as plt    # for plotting the images
%matplotlib inline
import pandas as pd
from keras.preprocessing import image   # for preprocessing the images
import numpy as np    # for mathematical operations
from keras.utils import np_utils
from skimage.transform import resize   # for resizing images
from sklearn.model_selection import train_test_split
from glob import glob
from tqdm import tqdm

from google_drive_downloader import GoogleDriveDownloader as gdd
from google.colab.patches import cv2_imshow

In [ ]:
!mkdir Train
!mkdir Test

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("dark")
plt.rcParams['figure.figsize'] = 16, 12
import pandas as pd
from PIL import Image
from tqdm import tqdm_notebook
from skimage import transform
import itertools as it
from sklearn.neighbors.kde import KernelDensity
import matplotlib.cm as cm
import queue
from skimage import morphology
import dlib
import cv2
import io
import time
from imutils import face_utils
from scipy.spatial import Delaunay

from scipy.signal import find_peaks
from scipy.signal import savgol_filter
from scipy.signal import medfilt
from scipy.interpolate import interp1d

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.kde module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


In [ ]:
# videos_train = [
#                 'https://drive.google.com/file/d/17Yx58VEiZivlbF8RWbe13VItCkUULoFk/view?usp=sharing',
#                 'https://drive.google.com/file/d/1mT2gXOWhkk_Mg7rYN7CRUwfgjTz-Dy1K/view?usp=sharing',
#                 'https://drive.google.com/file/d/1u5cb36lKSRlCf_ca8uPMfZMYKqTPB91k/view?usp=sharing',
#                 'https://drive.google.com/file/d/11ssqpN_ZnJrKmhf7hhmlw_tsyn74AaPI/view?usp=sharing',
#                 'https://drive.google.com/file/d/1Wdsw2f_y0dWzkfI7rZvDJjNL7_fzNJUm/view?usp=sharing',
#                 'https://drive.google.com/file/d/1iUMizR0WvmfBtSEAdH2lz4oU1Wy87THZ/view?usp=sharing',
#                 'https://drive.google.com/file/d/1UYSv2ebdH9lZJ6HHTeZY75iXD_gNMheB/view?usp=sharing'
# ]
# breathing_type = [1, 1, 0, 1, 1, 1, 0]

In [ ]:
# train = []
# for i, link in enumerate(videos_train):
#   id = link[link.find('/d/') + 3:link.find('/view')]
#   gdd.download_file_from_google_drive(file_id=id,
#                                       dest_path='./Train/train_{}.MOV'.format(i))
#   train.append({'video_name': 'train_{}.MOV'.format(i)})
# train = pd.DataFrame(train)
# train['tag'] = breathing_type

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip /content/drive/MyDrive/Test_vid.zip -d /content/Test
!unzip /content/drive/MyDrive/Train_vids.zip -d /content/Train

Archive:  /content/drive/MyDrive/Test_vid.zip
  inflating: /content/Test/Видео для теста.MOV  
Archive:  /content/drive/MyDrive/Train_vids.zip
  inflating: /content/Train/видео по 1 шаблону пульс 90-80.MOV  
  inflating: /content/Train/видео по 4 шаблону пульс 154.MOV  
  inflating: /content/Train/видос по 2 шаблону.MOV  
  inflating: /content/Train/пульс 116-90.MOV  
  inflating: /content/Train/пульс 142-108 3-4 шаблон.MOV  
  inflating: /content/Train/пульс 160-150 4 шаблон.MOV  
  inflating: /content/Train/пульс 90-80 видео по 2 шаблону.MOV  
  inflating: /content/Train/спокойное дыхание по 1 шаблону.MOV  
  inflating: /content/Train/спокойное дыхание пульс 90.MOV  


In [ ]:
# Видео на которых размечен пульс
videos_pulse = ['видео по 1 шаблону пульс 90-80',
                'видео по 4 шаблону пульс 154',
                'пульс 90-80 видео по 2 шаблону',
                'пульс 116-90',
                'пульс 142-108 3-4 шаблон',
                'пульс 160-150 4 шаблон',
                'спокойное дыхание пульс 90']

In [ ]:
# Делаем датафрейм под раскажровку пульса
df_targets = pd.DataFrame(columns=['video', 'heartbeat_times'])
for video in videos_pulse:
  cap = cv2.VideoCapture('/content/Train/' + video + '.MOV')
  fps = cap.get(cv2.CAP_PROP_FPS)      # OpenCV2 version 2 used "CV_CAP_PROP_FPS"
  frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
  heartbeat_times = np.arange(0, frame_count/fps, 1 / fps)
  empty_list = []
  for i in heartbeat_times:
    empty_list.append({'heartbeat_times': i})
  temp_df = pd.DataFrame(empty_list)
  temp_df['video'] = video
  try:
    df_targets = pd.concat([df_targets, temp_df])
  except: 
    pass

df_targets

,video,heartbeat_times
0,видео по 1 шаблону пульс 90-80,0.000000
1,видео по 1 шаблону пульс 90-80,0.033362
2,видео по 1 шаблону пульс 90-80,0.066724
3,видео по 1 шаблону пульс 90-80,0.100086
4,видео по 1 шаблону пульс 90-80,0.133448
...,...,...
2063,спокойное дыхание пульс 90,68.791606
2064,спокойное дыхание пульс 90,68.824952
2065,спокойное дыхание пульс 90,68.858297
2066,спокойное дыхание пульс 90,68.891642


In [ ]:
# Функцию разметки пульса делаем руками
def get_target_pulse(row):
  if row['video'] == 'видео по 1 шаблону пульс 90-80':
    return 1
  elif row['video'] == 'пульс 90-80 видео по 2 шаблону':
    return 1
  elif row['video'] == 'пульс 160-150 4 шаблон':
    return 4
  elif (row['video'] == 'видео по 4 шаблону пульс 154') & (row['heartbeat_times'] < 15):
    return 4
  elif (row['video'] == 'видео по 4 шаблону пульс 154') & (row['heartbeat_times'] >= 15):
    return 3
  elif (row['video'] =='пульс 116-90') & (row['heartbeat_times'] < 52):
    return 2
  elif (row['video'] =='пульс 116-90') & (row['heartbeat_times'] >= 52):
    return 1
  elif (row['video'] =='пульс 142-108 3-4 шаблон') & (row['heartbeat_times'] < 22):
    return 3
  elif (row['video'] =='пульс 142-108 3-4 шаблон') & (row['heartbeat_times'] >= 22):
    return 2
  elif (row['video'] == 'спокойное дыхание пульс 90') & (row['heartbeat_times'] < 18):
    return 1
  else:
    return 2

In [ ]:
# Определяем бин таргета
df_targets['target_bin'] = df_targets.apply(get_target_pulse, axis = 1)
df_targets.head(3)

,video,heartbeat_times,target_bin
0,видео по 1 шаблону пульс 90-80,0.000000,1
1,видео по 1 шаблону пульс 90-80,0.033362,1
2,видео по 1 шаблону пульс 90-80,0.066724,1


In [ ]:
# Вытаскиваем кадр относящийся к 5 секунде видео, чтобы быть уверенными что человек уже стоит
def sec_5_cadr(video, folder='Train/'):
  cap = cv2.VideoCapture('/content/' + folder + video + '.MOV')
  cap.set(cv2.CAP_PROP_POS_MSEC, 5000)      # just cue to 20 sec. position
  success, image = cap.read()
  return image

In [ ]:
!wget http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
!tar xvjf shape_predictor_68_face_landmarks.dat.bz2
!bunzip2 /content/shape_predictor_68_face_landmarks.dat.bz2

--2021-03-27 02:45:19--  http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
Resolving dlib.net (dlib.net)... 107.180.26.78
Connecting to dlib.net (dlib.net)|107.180.26.78|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 64040097 (61M)
Saving to: ‘shape_predictor_68_face_landmarks.dat.bz2’

shape_predictor_68_ 100%[===================>]  61.07M  15.0MB/s    in 6.4s    

2021-03-27 02:45:26 (9.54 MB/s) - ‘shape_predictor_68_face_landmarks.dat.bz2’ saved [64040097/64040097]

tar: This does not look like a tar archive
tar: Skipping to next header
tar: Exiting with failure status due to previous errors


In [ ]:
# Функция выделения на видео
def get_bbox(image):
  detector = dlib.get_frontal_face_detector()
  # инстанцируем класс для детекции ключевых точек
  predictor = dlib.shape_predictor('/content/shape_predictor_68_face_landmarks.dat')

  img = image
  # конвертируем изображение в много оттенков серого
  img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  # вычисляем список рамок на каждое найденное лицо
  rects = detector(img_gray, 0)
  # вычисляем ключевые точки
  shape = predictor(img_gray, rects[0])
  shape = face_utils.shape_to_np(shape)

  # оригинальная рамка
  face_origin = sorted([(t.width()*t.height(), 
                        (t.left(), t.top(), t.width(), t.height())) 
                        for t in rects], 
                      key=lambda t: t[0], reverse=True)[0][1]
  return face_origin

In [ ]:
# Функция распознавания пульса
def predict_pulse(video, face_origin, df_to_be_extended, folder='Train/'):
  cap = cv2.VideoCapture('/content/' + folder + video + '.MOV')
  cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1920)
  cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 1080)
  cap.set(cv2.CAP_PROP_FPS, 30)

  heartbeat_count = 128
  heartbeat_values = [0]*heartbeat_count
  heartbeat_times = [0]*heartbeat_count

  while(True):
      ret, frame = cap.read()

      # Our operations on the frame come here
      try:
        img = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
      except:
        break
      x, y, w, h = face_origin
      crop_img = img[y:y + h, x:x + w]

      # Update the list
      heartbeat_values = heartbeat_values[:] + [np.average(crop_img)]
      heartbeat_times = heartbeat_times[:] + [heartbeat_times[-1]+1/30]

  cap.release()
  cv2.destroyAllWindows()

  # Отсекаем начальные значения
  heartbeat_values1 = np.array([val for val in heartbeat_values if val !=0])
  heartbeat_times1 = heartbeat_times[len(heartbeat_values)-len(heartbeat_values1):]

  #Определяем пульс
  x = savgol_filter(heartbeat_values1, 5, polyorder=2)
  x = medfilt(x, 3)
  peaks, _= find_peaks(x, distance=10)

  pulse = []
  for i in range(len(peaks)-1):
    pulse.append(1800 / (peaks[i+1]-peaks[i]))
  pulse = medfilt(pulse, 15)
  pulse = np.insert(pulse, 0, pulse.mean())
  f = interp1d(peaks, pulse, kind='slinear')
  per = np.arange(peaks.min(), peaks.max())
  x_new = f(per)


  # Дописываем датафрейм
  empty_list = []
  for i,j in zip(x_new, per / 30):
    empty_list.append({'heartbeat_values_model': i, 'heartbeat_times_model': j})
  temp_df = pd.DataFrame(empty_list)
  temp_df['video'] = video

  try:
    df_to_be_extended = pd.concat([df_to_be_extended, temp_df])
  except: 
    pass
  
  return df_to_be_extended

In [ ]:
# Определяем пульс по видео, где он был размечен
df_to_be_extended = pd.DataFrame(columns=['heartbeat_values_model', 'heartbeat_times_model', 'video'])

for video in videos_pulse:
  image = sec_5_cadr(video)
  face_origin = get_bbox(image)
  df_to_be_extended = predict_pulse(video, face_origin, df_to_be_extended)

df_to_be_extended.head(3)

,heartbeat_values_model,heartbeat_times_model,video
0,79.362904,0.633333,видео по 1 шаблону пульс 90-80
1,76.655282,0.666667,видео по 1 шаблону пульс 90-80
2,73.947661,0.700000,видео по 1 шаблону пульс 90-80


In [ ]:
# Функция бинерезации
def get_bin(row):
  if row['heartbeat_values_model'] < 90:
    return 1
  elif row['heartbeat_values_model'] >= 90 and row['heartbeat_values_model'] < 120:
    return 2
  elif row['heartbeat_values_model'] >= 120 and row['heartbeat_values_model'] < 150:
    return 3
  else:
    return 4

In [ ]:
# Переводим предсказания в бины
df_to_be_extended['predicted_bin'] = df_to_be_extended.apply(get_bin, axis=1)
df_to_be_extended.head(3)

,heartbeat_values_model,heartbeat_times_model,video,predicted_bin
0,79.362904,0.633333,видео по 1 шаблону пульс 90-80,1
1,76.655282,0.666667,видео по 1 шаблону пульс 90-80,1
2,73.947661,0.700000,видео по 1 шаблону пульс 90-80,1


In [ ]:
df_to_be_extended.to_excel('Pulse.xlsx')

In [ ]:
# Округляем моменты измерения пульса
df_targets['heartbeat_times'] = df_targets['heartbeat_times'].apply(lambda x: np.round(x, 3))
df_targets.head(3)

,video,heartbeat_times,target_bin
0,видео по 1 шаблону пульс 90-80,0.000,1
1,видео по 1 шаблону пульс 90-80,0.033,1
2,видео по 1 шаблону пульс 90-80,0.067,1


In [ ]:
# Округляем моменты измерения пульса
df_to_be_extended['heartbeat_times_model'] = \
df_to_be_extended['heartbeat_times_model'].apply(lambda x: np.round(x, 3))
df_to_be_extended.head(3)

,heartbeat_values_model,heartbeat_times_model,video,predicted_bin
0,79.362904,0.633,видео по 1 шаблону пульс 90-80,1
1,76.655282,0.667,видео по 1 шаблону пульс 90-80,1
2,73.947661,0.700,видео по 1 шаблону пульс 90-80,1


In [ ]:
df_targets[df_targets['heartbeat_times'] == 0.7]

,video,heartbeat_times,target_bin
21,пульс 142-108 3-4 шаблон,0.7,3
21,спокойное дыхание пульс 90,0.7,1


In [ ]:
# Соедениям предсказанный и размеченный таргет
df = df_to_be_extended.merge(df_targets, how = 'left', left_on=['video', 'heartbeat_times_model'],
                             right_on = ['video', 'heartbeat_times'])
df.head(3)

,heartbeat_values_model,heartbeat_times_model,video,predicted_bin,heartbeat_times,target_bin
0,79.362904,0.633,видео по 1 шаблону пульс 90-80,1,NaN,NaN
1,76.655282,0.667,видео по 1 шаблону пульс 90-80,1,0.667,1.0
2,73.947661,0.700,видео по 1 шаблону пульс 90-80,1,NaN,NaN


In [ ]:
# На всякий случай заполнеие NA
df['heartbeat_times'] = df['heartbeat_times'].fillna(df['heartbeat_times_model'])
df['target_bin'] = df.apply(get_target_pulse, axis = 1)

In [ ]:
# Расчитаем взвешенную Accuracy
from sklearn.metrics import balanced_accuracy_score
balanced_accuracy_score(df['target_bin'], df['predicted_bin'])

0.33297363112234585

In [ ]:
# Предсказываем для тестового видео
test_df = pd.DataFrame(columns=['heartbeat_values_model', 'heartbeat_times_model', 'video'])

image = sec_5_cadr('Видео для теста', 'Test/')
face_origin = get_bbox(image)
test_df = predict_pulse('Видео для теста', face_origin, test_df, 'Test/')

test_df

,heartbeat_values_model,heartbeat_times_model,video
0,98.250719,1.433333,Видео для теста
1,96.910421,1.466667,Видео для теста
2,95.570124,1.500000,Видео для теста
3,94.229826,1.533333,Видео для теста
4,92.889528,1.566667,Видео для теста
...,...,...,...
2195,41.559819,74.600000,Видео для теста
2196,41.429673,74.633333,Видео для теста
2197,41.299528,74.666667,Видео для теста
2198,41.169382,74.700000,Видео для теста


Нашли теоретическую формулу для связи дыхания и пульса http://disser.ru/library/35/260.htm

In [ ]:
# Частота дыхания
test_df['breath_rate'] = (test_df['heartbeat_values_model'] - 48.516) / 0.8927
test_df.head(3)

,heartbeat_values_model,heartbeat_times_model,video,breath_rate
0,98.250719,1.433333,Видео для теста,55.712691
1,96.910421,1.466667,Видео для теста,54.211293
2,95.570124,1.500000,Видео для теста,52.709895
